# Interactive Mode with Remote Objects 
We can access the trainer class on remote, and work with it interactively *while* training is still happening. For instance, we might want to test out inference from local on the remote model as it trains. 

Runhouse gives researchers and ML engineers more options and powerful tools to interact with their workflows. 

In [23]:
import runhouse as rh 

# Connect to the cluster by name - possible only when using Runhouse Den to save resources
gpu_cluster = rh.cluster(
        name= 'py-2x1GPU')
gpu_cluster.is_up()


INFO | 2024-11-01 14:27:52 | runhouse.resources.hardware.cluster:1511 | Running command on py-2x1GPU: echo "hello"


True

In [27]:
# Grab the remote model trainer (0th worker) from the cluster
remote_model = gpu_cluster.get('train_1', default=None, remote=True)

In [21]:
import requests
from PIL import Image
from io import BytesIO

# URL of the image -  Aaron Judge
image_url = "https://pyxis.nymag.com/v1/imgs/181/899/caa5404cd8a7bf2bd3e5952a1e74da29ed-aaron-judge-yankees.1x.rsquare.w1400.jpg"

# Fetch the image from the URL
response = requests.get(image_url)

# Ensure the request was successful
if response.status_code == 200:
    # Create a PIL image from the response content
    image = Image.open(BytesIO(response.content))
else:
    print(f"Failed to retrieve the image. Status code: {response.status_code}")


Now we can use the remote model (as training is happpening) to see if it correctly predicts ballplayer

In [28]:
remote_model.predict(image)

INFO | 2024-11-01 14:28:36 | runhouse.servers.http.http_client:432 | Calling train_1.predict
---------
py-2x1GPU
---------
ERROR | 2024-11-01 14:28:41 | runhouse.servers.http.http_utils:311 | Error calling predict on train_1 on server: The actor died unexpectedly before finishing this task.
ERROR | 2024-11-01 14:28:41 | runhouse.servers.http.http_utils:311 | 	class_name: EnvServlet
ERROR | 2024-11-01 14:28:41 | runhouse.servers.http.http_utils:311 | 	actor_id: de0e13b4822dd3abf8e977a901000000
ERROR | 2024-11-01 14:28:41 | runhouse.servers.http.http_utils:311 | 	pid: 38112
ERROR | 2024-11-01 14:28:41 | runhouse.servers.http.http_utils:311 | 	name: pytorch_env_1
ERROR | 2024-11-01 14:28:41 | runhouse.servers.http.http_utils:311 | 	namespace: runhouse
ERROR | 2024-11-01 14:28:41 | runhouse.servers.http.http_utils:311 | 	ip: 172.31.85.225
ERROR | 2024-11-01 14:28:41 | runhouse.servers.http.http_utils:311 | The actor is dead because its worker process has died. Worker exit type: SYSTEM_ERRO

ActorDiedError: The actor died unexpectedly before finishing this task.
	class_name: EnvServlet
	actor_id: de0e13b4822dd3abf8e977a901000000
	pid: 38112
	name: pytorch_env_1
	namespace: runhouse
	ip: 172.31.85.225
The actor is dead because its worker process has died. Worker exit type: SYSTEM_ERROR Worker exit detail: Worker unexpectedly exits with a connection error code 2. End of file. There are some potential root causes. (1) The process is killed by SIGKILL by OOM killer due to high memory usage. (2) ray stop --force is called. (3) The worker is crashed unexpectedly due to SIGSEGV or other unexpected errors.

In [26]:
remote_model